So what if we started at a HACK emulator

Ainstr
Cinstr


ImpBV - imp with only bitvectors.
bits - parts


Super optimization. I can encode instructions easily to Z3 as enums

erasing PC is a big step. partial evaluation?

rom as a z3 array vs not. I suppose it has to be a Z3 array biascally.

I baulked at the idea of using a dict for my switch case. But there is a version of the z3 interpreter that does this.
My if then esle have to become z3 ite(). brutal.

So you really want to partially evaluate that.  That's really interesting that this is the relationship of the PC aware version and the non PC aware version is partialk evaluation. we stage an interpeter.

Can I make this work with nmigen?

That riscv cpu https://news.ycombinator.com/item?id=27575174

Could also attempt a BF interpeter
or FORTH?
WASM?
AVR

Could I use fetch as my symbolic execution point?


I could easily translate this spec to Coq.

There is a debate if we hsould take a unop/binop/const breakdown of computations. Is this easier?

Const = 
UnOp = Id, Not, Neg, Inc, Dec
BinOp = Add, Sub Or, And,, not all combos avaianle? nope. 

Dest = One Two Three.

It's all not worth it.


We could encode evertything into Z3
We could meta program into Z3kanren... That's intriguing. but fetch...

How can I write my interpeter such that it works in all these different situations.
python control flow constructs and assignement are insufficiently generic

It is somewhat interesting that there is some relation of my difficulties here and coq functions vs relations.
If i need to run backwards or reinterpety through z3 it feels like I'm sunk.
What is the difference between a coq relation and a dict? Ok dict is functional
Or [( Jump.Op , a + b )] . This is a value levbel relation.


We can build basic blocks. Runs of Null.
And we do have direct jumps. JMP.



SMT encoding
Work at draper on veryfing binaries
Typelevel/Value vs compiletime/runtime

take what we get vs desgin from scratch

What does specification mean?

Julia? Enough macro magic to do what you need.
nmgien?
My old python catgoery blog posts - graphviz, z3, etc.


haskell to agda

pepem keynote at popl - turing 2019

systolic
skewed data
arrow catregories - comma catgeoies

deriving compostion from a functor. You have the functor but 

hmormohpism has nothing to do with laws


symbolic execution.
Seperate the 



bisimulation
Write Z3 interpreter of hack
Write Z3 interpreter of VM
Write state correspondence





In [ ]:
Seg = Enum('Segments', '''LOCAL ARGUMENTS STATIC CONSTANT''')
ACom = Enum('ArithCommands', '''add sub neg eq gt lt and or not''')

@dataclass
class Push:
   segment: Seg
   loc: int
@dataclass
class Pop:
   segment: Seg
   loc: int




In [81]:
from dataclasses import dataclass
from enum import Enum       
import typing
from typing import List, Union

Comp = Enum('Comp', '''ZERO ONE NEGONE D A M NOTD NOTA NOTM NEGD NEGA NEGM INCD INCA INCM DECD DECA DECM 
                         ADD_DA ADD_DM SUB_DA SUB_DM SUB_AD SUB_MD AND_DA AND_DM OR_DA OR_DM''') 
Dest = Enum('Dest', 'NULL M D MD A AM AD AMD') 
Jump = Enum('Jump', 'NULL JGT JEQ JGE JLT JNE JLE JMP') 

@dataclass
class AInsn:
    lit: int

@dataclass
class CInsn:
    dest: Dest
    comp: Comp
    jump: Jump

@dataclass
class HackState:
    rA: int
    rD: int
    pc: int
    mem: List[int]

def execute(instr : CInsn, s : HackState):
    A = s.rA
    D = s.rD
    pc = s.pc
    M = s.mem[A]
    
    if instr.comp == Comp.ZERO:
        comp = 0
    elif instr.comp == Comp.ONE:
        comp = 1
    elif instr.comp == Comp.NEGONE:
        comp = -1
    elif instr.comp == Comp.D:
        comp = D
    elif instr.comp == Comp.A:
        comp = A
    elif instr.comp == Comp.M:
        comp = M
    elif instr.comp == Comp.NOTD:
        comp = ~D
    elif instr.comp == Comp.NOTA:
        comp = ~A
    elif instr.comp == Comp.NOTM:
        comp = ~M
    elif instr.comp == Comp.NEGD:
        comp = -D
    elif instr.comp == Comp.NEGA:
        comp = -A
    elif instr.comp == Comp.NEGM:
        comp = -M
    elif instr.comp == Comp.INCD:
        comp = D + 1
    elif instr.comp == Comp.INCA:
        comp = A + 1
    elif instr.comp == Comp.INCM:
        comp = M + 1
    elif instr.comp == Comp.DECD:
        comp = D - 1
    elif instr.comp == Comp.DECA:
        comp = A - 1
    elif instr.comp == Comp.DECM:
        comp = M - 1
    elif instr.comp == Comp.ADD_DA:
        comp = D + A
    elif instr.comp == Comp.ADD_DM:
        comp = D + M
    elif instr.comp == Comp.SUB_DA:
        comp = D - A
    elif instr.comp == Comp.SUB_DM:
        comp = D - M
    elif instr.comp == Comp.SUB_AD:
        comp = A - D
    elif instr.comp == Comp.SUB_MD:
        comp = M - D
    elif instr.comp == Comp.AND_DA:
        comp = D & A
    elif instr.comp == Comp.AND_DM:
        comp = D & M
    elif instr.comp == Comp.OR_DA:
        comp = D | S
    elif instr.comp == Comp.OR_DM:
        comp = D | M
    else:
        assert False
    
    if instr.dest == Dest.NULL:
        pass
    elif instr.dest == Dest.M:
        M = comp
    elif instr.dest == Dest.D:
        D = comp
    elif instr.dest == Dest.MD:
        M = comp
        D = comp
    elif instr.dest == Dest.A:
        A = comp
    elif instr.dest == Dest.AM:
        A = comp
        M = comp
    elif instr.dest == Dest.AD:
        A = comp
        D = comp
    elif instr.dest == Dest.AMD:
        A = comp
        M = comp
        D = comp
    else:
        assert False
        
    if instr.jump == Jump.NULL:
        jump = False
    elif instr.jump == Jump.JGT:
        jump = comp > 0
    elif instr.jump == Jump.JEQ:
        jump = comp == 0
    elif instr.jump == Jump.JGE:
        jump = comp >= 0
    elif instr.jump == Jump.JLT:
        jump = comp < 0
    elif instr.jump == Jump.JNE:
        jump = comp != 0
    elif instr.jump == Jump.JLE:
        jump = comp <= 0
    elif instr.jump == Jump.JMP:
        jump = True
    else:
        assert False
    
    # we may not want to mutate here.
    # return comp, jump, dest instead?
    if jump:
        s.pc = s.rA
    else:
        s.pc = pc + 1
    
    s.mem[s.rA] = M #??? What is the order here?
    s.rA = A
    s.rD = D

    # return A, D, M, jump ?
    return s

Insn = Union[AInsn,CInsn]

def fetch(rom : List[Instr], s : HackState) -> Insn:
    return rom[s.pc]


def run(rom):
    # hmm. We should make 16 bit vectors.
    state = HackState(0,0,0,[0]*2**13)
    while state.pc != -1:
        instr = fetch(rom, state)
        state = execute(instr, state)
    return state


In [ ]:
def step(rom, state):
    instr = fetch(rom, state)
    state = execute(instr, state)
    return state
    

In [93]:
#Comp = Enum('Comp', '''ZERO ONE NEGONE D A M NOTD NOTA NOTM NEGD NEGA NEGM INCD INCA INCM DECD DECA DECM 
#                         ADD_DA ADD_DM SUB_DA SUB_DM SUB_AD SUB_MD AND_DA AND_DM OR_DA OR_DM''') 
#Dest = Enum('Dest', 'NULL M D MD A AM AD AMD') 
#Jump = Enum('Jump', 'NULL JGT JEQ JGE JLT JNE JLE JMP') 

#Tables from Chpater 5 slide 33
comp_table = {
    0b0101010 : Comp.ZERO,
    0b0111111 : Comp.ONE,
    0b0111010 : Comp.NEGONE,
    0b0001100 : Comp.D,
    0b0110000 : Comp.A,
    0b1110000 : Comp.M,
    0b0110001 : Comp.NOTA,
    0b1110001 : Comp.NOTM,
    0b0001111 : Comp.NEGD,
    0b0110011 : Comp.NEGA,
    0b1110011 : Comp.NEGM,
    0b0011111 : Comp.INCD,
    0b0110111 : Comp.INCA,
    0b1110111 : Comp.INCM,
    0b0001110 : Comp.DECD,
    0b0110010 : Comp.DECA,
    0b1110010 : Comp.DECM,
    0b0000010 : Comp.ADD_DA,
    0b1000010 : Comp.ADD_DM,
    0b0010011 : Comp.SUB_DA,
    0b1010011 : Comp.SUB_DM,
    0b0000111 : Comp.SUB_AD,
    0b1000111 : Comp.SUB_MD,
    0b0000000 : Comp.AND_DA,
    0b1000000 : Comp.AND_DM,
    0b0010101 : Comp.OR_DA,
    0b1010101 : Comp.OR_DM
    
}


jmp_table = {
    0b000 : Jump.NULL,
    0b001 : Jump.JGT,
    0b010 : Jump.JEQ,
    0b011 : Jump.JGE,
    0b100 : Jump.JLT,
    0b101 : Jump.JNE,
    0b110 : Jump.JLE,
    0b111 : Jump.JMP
}

dest_table = {
    0b000 : Dest.NULL,
    0b001 : Dest.M,
    0b010 : Dest.D,
    0b011 : Dest.MD,
    0b100 : Dest.A,
    0b101 : Dest.AM,
    0b110 : Dest.AD,
    0b111 : Dest.AMD
}


def lift_insn(insn):
    assert(0 <= insn < 2**16)
    jmp = 0b111 & insn
    dest = 0b111 & (insn >> 3)
    comp = 0b111111 & (insn >> 6)
    
    is_C_insn = (insn >> 15) == 1
    if is_C_insn:
        return CInsn(dest_table[dest], comp_table[comp], jmp_table[jmp])
    else:
        return AInsn(insn)
    

def disasm(text):
    return [lift_insn(int(insn,2)) for insn in text.split("\n")]
    

assert(lift(1) == Ainstr(1))
lift(0b1000000000000001)
lift(0b100_0000000_011_000)

prog = disasm(
'''0000000000001111
1110110000010000
1110011111001000
1111110111100000
1110110010010000
0000000000000000
1110101010000111''')

#for i in range(2**16):
#    lift(i)
prog

[AInsn(lit=15),
 CInsn(dest=<Dest.D: 3>, comp=<Comp.A: 5>, jump=<Jump.NULL: 1>),
 CInsn(dest=<Dest.M: 2>, comp=<Comp.INCD: 13>, jump=<Jump.NULL: 1>),
 CInsn(dest=<Dest.A: 5>, comp=<Comp.INCA: 14>, jump=<Jump.NULL: 1>),
 CInsn(dest=<Dest.D: 3>, comp=<Comp.DECA: 17>, jump=<Jump.NULL: 1>),
 AInsn(lit=0),
 CInsn(dest=<Dest.NULL: 1>, comp=<Comp.ZERO: 1>, jump=<Jump.JMP: 8>)]

In [97]:
def jump_table(comp):
    return { 
    Jump.NULL: False,
    Jump.JGT: comp > 0,
    Jump.JEQ: comp == 0,
    Jump.JGE: comp >= 0,
    Jump.JLT: comp < 0,
    Jump.JNE: comp != 0,
    Jump.JLE: comp <= 0,
    Jump.JMP: True
    }
jump_table(7)
def comp_table(s):
    A = s.rA
    D = s.rD
    pc = s.pc
    M = s.mem(A)
    return {
        Comp.ZERO:  0,
        Comp.ONE: 1,
        Comp.NEGONE: -1,
        Comp.D: D,
        Comp.A: A,
        Comp.M: M,
        Comp.NOTD: ~D,
        Comp.NOTA: ~A,
        Comp.NOTM: ~M,
        Comp.NEGD: -D,
        Comp.NEGA: -A,
        Comp.NEGM: -M,
        Comp.INCD: D + 1,
        Comp.INCA: A + 1,
        Comp.INCM: M + 1,
        Comp.DECD: D - 1,
        Comp.DECA: A - 1,
        Comp.DECM: M - 1,
        Comp.ADD_DA: D + A,
        Comp.ADD_DM: D + M,
        Comp.SUB_DA: D - A,
        Comp.SUB_DM: D - M,
        Comp.SUB_AD: A - D,
        Comp.SUB_MD: M - D,
        Comp.AND_DA: D & A,
        Comp.AND_DM: D & M,
        Comp.OR_DA: D | A,
        Comp.OR_DM: D | M
    }





In [125]:
from z3 import *
BV16 = BitVecSort(16)
BV8 = BitVecSort(8)
A, D, PC, A1, D1, PC1 = BitVecs("A D PC A1 D1 PC1", BV16)
ramsize = 8
MEM = Array("RAM", BitVecSort(ramsize), BV16)
MEM1 = Array("RAM1", BitVecSort(ramsize), BV16)
RAM = lambda addr: MEM[Extract(ramsize,1,addr)]
RAM1 = lambda addr: MEM1[Extract(ramsize,1,addr)]
st = HackState(A,D,PC,RAM)

M = RAM(A)
M1 = RAM1(A1)


z3_comp_table = comp_table(st)

comp = BitVec("comp", BV16)
z3_jump_table = jump_table(comp)


z3_dest_table = dest_table = {
    Dest.NULL : [],
    Dest.M : [],
    Dest.D : [D1],
    Dest.MD : [D1],
    Dest.A : [A1],
    Dest.AM : [A1],
    Dest.AD : [A1,D1],
    Dest.AMD : [A1,D1]
}

Insn = Function("Insn", BV16, BV16, BV16, ArraySort(BV8, BV16), BoolSort())
Implies(Insn(A,D,PC,MEM), Insn(A1,D1,PC1,MEM1))
def z3_insn(insn):
    if type(insn) == AInsn:
        return And([A1 == BitVecVal(insn.lit, 16), 
                    PC1 == PC + 1, 
                    D == D1, 
                    MEM == MEM1])
    elif type(insn) == CInsn:
        Trans = []

        comp1 = z3_comp_table[insn.comp]
        #newvals = [dest == comp1 for dest in [A1,D1] z3_dest_table[insn.dest]]
        if insn.dest in [Dest.A, Dest.AD, Dest.AD, Dest.AMD]:
            Trans += [A1 == comp1]
        else:
            Trans += [A1 == A]
        if insn.dest == Dest.D or insn.dest == Dest.MD or insn.dest == Dest.AD or insn.dest == Dest.AMD:
            Trans += [D1 == comp1]
        else:
            Trans += [D1 == D]
        if insn.dest == Dest.M or insn.dest == Dest.MD or insn.dest == Dest.AM or insn.dest == Dest.AMD:
            Trans += [MEM1 == Store(MEM, Extract(ramsize,1,A), comp1)]
        else:
            Trans += [MEM1 == MEM]
        
        cond = z3_jump_table[insn.jump]
        Trans += [If(cond, PC1 == A, PC1 == PC + 1)]
        
        return And(Trans)
        #return Implies(And(Insn(A1,D1,PC1,MEM1), Trans), Insn(A,D,PC,MEM))

[z3_insn(insn) for insn in prog]      

[And(15 == A1, PC1 == PC + 1, D == D1, RAM == RAM1),
 And(A1 == A,
     D1 == A,
     RAM1 == RAM,
     If(False, PC1 == A, PC1 == PC + 1)),
 And(A1 == A,
     D1 == D,
     RAM1 == Store(RAM, Extract(8, 1, A), D + 1),
     If(False, PC1 == A, PC1 == PC + 1)),
 And(A1 == A + 1,
     D1 == D,
     RAM1 == RAM,
     If(False, PC1 == A, PC1 == PC + 1)),
 And(A1 == A,
     D1 == A - 1,
     RAM1 == RAM,
     If(False, PC1 == A, PC1 == PC + 1)),
 And(0 == A1, PC1 == PC + 1, D == D1, RAM == RAM1),
 And(A1 == A,
     D1 == D,
     RAM1 == RAM,
     If(True, PC1 == A, PC1 == PC + 1))]

In [176]:
#https://ericpony.github.io/z3py-tutorial/fixpoint-examples.htm
# https://github.com/agurfinkel/spacer-on-jupyter/blob/master/Dagstuhl2019.ipynb
fp = Fixedpoint()
fp.declare_var(A, D, PC, A1, D1, PC1, MEM, MEM1)
fp.register_relation(Insn)
for pc,insn in enumerate(prog):
    Trans = And(z3_insn(insn), PC == pc)
    fp.rule(Insn(A,D,PC,MEM), And(Trans, Insn(A1,D1,PC1,MEM1)))

# Hmm. Inital and final state?
fp.rule(Insn(A,D,PC,MEM), PC >= 10)
fp.query(And(Insn(A,D,PC,MEM), PC == 0))
fp.get_answer()

# unsat, as in the prolog search will never find a solution.
# Yes. Ok. So this is a prolog search for a bad state starting from PC = 0.


ForAll([A, B, C, D],
       Insn(A, B, C, D) ==
       And(Not(C == 3),
           Not(C == 2),
           Or(Extract(1, 1, C) == 1, 10 <= C),
           Or(Not(A == 0), 10 <= C)))



reachable(A,D,0,MEM) :- reachable(15,D,1,MEM).
reachable(A,D,1,MEM) :- reachable( A ,A + D,2,).
reachable(A,D,2,MEM) :- reachable( A ,A + D,2,).
reachable( A ,A + D,2,) :- reachable( A ,A + D,2,).


In [162]:
s = SolverFor("HORN")
for pc, insn in enumerate(prog):
    Trans = And(z3_insn(insn), PC == pc)
    s.add(ForAll([A,A1,D,D1,PC,PC1,MEM,MEM1], 
                 Implies(And(Trans, Insn(A,D,PC,MEM)),
                         Insn(A1,D1,PC1,MEM1))))
Init = PC == 0
s.add(ForAll([A,A1,D,D1,PC,PC1,MEM,MEM1], Implies(Init, Insn(A,D,PC,MEM))))
s.add(ForAll([A, D, PC, MEM], Not(And(Insn(A,D,PC,MEM), PC >= 7) )))
print(s.check())
s.model()

sat


[Insn = [else ->
         And(Not(7 <= Var(2)),
             Or(Not(Var(2) == 6), Not(7 <= Var(0))))]]

So from a backwards chaining persepctuive, the query is
Insn(A,D,PC,MEM), PC >= 7

And it chains backwards throuh the program

Right. So "Principle of Backward induction" Is a forward simulating prolog program



In [174]:
s = SolverFor("HORN")
for pc, insn in enumerate(prog):
    Trans = And(z3_insn(insn), PC == pc)
    s.add(ForAll([A,A1,D,D1,PC,PC1,MEM,MEM1], 
                 Implies(And(Trans, Insn(A1,D1,PC1,MEM1) ),
                         Insn(A,D,PC,MEM))))
Init = PC == 0

s.add(ForAll([A,A1,D,D1,PC,PC1,MEM,MEM1], Implies(PC >= 7, Insn(A,D,PC,MEM))))
s.add(ForAll([A, D, PC, MEM], Not(And(Insn(A,D,PC,MEM), Init) )))
print(s.check())
s.model()
#s

sat


[Insn = [else ->
         And(Or(Not(Var(0) == 0), 7 <= Var(2)),
             Not(Var(2) == 0),
             Not(Var(2) == 4),
             Or(Not(Extract(0, 0, Var(2)) == 1),
                7 <= Var(2)),
             Not(Var(2) == 2))]]

In [ ]:
import z3

D = BitVec()

z3_comp_tables = 



    

In [153]:
# Two analgous programs for the mc problem
Z = IntSort()
B = BoolSort()
s = SolverFor("HORN")
mc = Function('mc', Z, Z, B)
x, y, z = Ints('x y z')
s.add(ForAll(x, Implies(x > 100, mc(x, x - 10))))
s.add(ForAll([x, y, z], 
             Implies(And(x <= 100, mc(x + 11, y), mc(y, z)), 
                     mc(x, z))))
s.add(ForAll([x, y], Implies(And(x <= 101, mc(x, y)), y == 91)))
s.add(ForAll([x, y], Implies(And(x >= 101, mc(x, y)), x == y + 10)))
print(s.check())
s.model()

sat


[mc = [else ->
       And(Or(Not(Var(1) >= 92),
              Not(Var(1) + -1*Var(0) >= -9)),
           Not(Var(1) <= 90),
           Not(Var(1) + -1*Var(0) <= -11))]]

In [ ]:
mc = Function('mc', IntSort(), IntSort(), BoolSort())
n, m, p = Ints('n m p')

fp = Fixedpoint()

fp.declare_var(n,m)
fp.register_relation(mc)

fp.rule(mc(m, m-10), m > 100)
fp.rule(mc(m, n), [m <= 100, mc(m+11,p),mc(p,n)])
    
print fp.query(And(mc(m,n),n < 90))
print fp.get_answer()

print fp.query(And(mc(m,n),n < 91))
print fp.get_answer()

print fp.query(And(mc(m,n),n < 92))
print fp.get_answer()

In [ ]:
def interp_computation(instr : Cinstr, s : HackState):
    A = s.rA
    D = s.rD
    pc = s.pc
    M = s.mem[A]
    
    if instr.comp == Comp.ZERO:
        comp = 0
    elif instr.comp == Comp.ONE:
        comp = 1
    elif instr.comp == Comp.NEGONE:
        comp = -1
    elif instr.comp == Comp.D:
        comp = D
    elif instr.comp == Comp.A:
        comp = A
    elif instr.comp == Comp.M:
        comp = M
    elif instr.comp == Comp.NOTD:
        comp = ~D
    elif instr.comp == Comp.NOTA:
        comp = ~A
    elif instr.comp == Comp.NOTM:
        comp = ~M
    elif instr.comp == Comp.NEGD:
        comp = -D
    elif instr.comp == Comp.NEGA:
        comp = -A
    elif instr.comp == Comp.NEGM:
        comp = -M
    elif instr.comp == Comp.INCD:
        comp = D + 1
    elif instr.comp == Comp.INCA:
        comp = A + 1
    elif instr.comp == Comp.INCM:
        comp = M + 1
    elif instr.comp == Comp.DECD:
        comp = D - 1
    elif instr.comp == Comp.DECA:
        comp = A - 1
    elif instr.comp == Comp.DECM:
        comp = M - 1
    elif instr.comp == Comp.ADD_DA:
        comp = D + A
    elif instr.comp == Comp.ADD_DM:
        comp = D + M
    elif instr.comp == Comp.SUB_DA:
        comp = D - A
    elif instr.comp == Comp.SUB_DM:
        comp = D - M
    elif instr.comp == Comp.SUB_AD:
        comp = A - D
    elif instr.comp == Comp.SUB_MD:
        comp = M - D
    elif instr.comp == Comp.AND_DA:
        comp = D & A
    elif instr.comp == Comp.AND_DM:
        comp = D & M
    elif instr.comp == Comp.OR_DA:
        comp = D | S
    elif instr.comp == Comp.OR_DM:
        comp = D | M
    else:
        assert False
    return comp


def interp_dest(instr : Cinstr, s : HackState):
    A = s.rA
    D = s.rD
    pc = s.pc
    M = s.mem[A]

    if instr.dest == Dest.NULL:
        pass
    elif instr.dest == Dest.M:
        M = comp
    elif instr.dest == Dest.D:
        D = comp
    elif instr.dest == Dest.MD:
        M = comp
        D = comp
    elif instr.dest == Dest.A:
        A = comp
    elif instr.dest == Dest.AM:
        A = comp
        M = comp
    elif instr.dest == Dest.AD:
        A = comp
        D = comp
    elif instr.dest == Dest.AMD:
        A = comp
        M = comp
        D = comp
    else:
        assert False
        
    s = copy(s)
    s.mem[s.rA] = M #??? What is the order here?
    s.rA = A
    s.rD = D
    return s


    

In [ ]:
def z3_switch(x, d):
    acc = BoolVal(True)
    for k,v in d.items()
        acc = If(x == k, v, acc)
    return acc

# if instead of using extended if elif cases. I had used the dicctionary method
# lots of unnecessary computation. but more directly overloadable.

def switch(x,d):
    for k,v in d.items():
        if x == k:
            return v

In [ ]:
def z3_fetch(rom, state):
    while s.check():
        s.push()
        m = s.getmodel()
        mpc = m[state.pc].as_long()
        yield rom[mpc]
        s.add(state.pc != mpc)



Abstract interpetation to figure out CFG ahead of time. otherwise we have trouble.

A D PC = BitSet(2^16)
Not totally insane?
Mem is insane though.


https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=8805076

It should be extremely rare there are many addresses it can jump to
And it is in fact limitted by the small program memory space.


I suspect that PC constraints can be dealth with similarly to invaraints. User can specify what possibilities and that is easy enough to verify.


In [8]:
x = set()
type(x) is set

True

In [ ]:
class SetDomain():
    def __init__(self, x):
        if type(x) is set:
            self.x = x
        elif type(x) is SetDomain:
            self.x = x.x
        else:
            self.x = set(x)
    def __add__(self,b):
        set([ x + y for x in self.x for y in b.x ])
    
    
        

In [ ]:
def alu(instr, HackState):
    If(instr == 0, )

In [3]:
class VSA():
    def __init__(self, ):
        
    def __add__():
        
        

8192

"Everything is a loop" in assembly

That's an interesting mantra. And we need invariants annotated everywhere
And straight line if the else structures are the exception rather than the rule. That allow us to propagate state summaries

An idea:
A block has multiple entries and exits. but is otherwise sitraght control flow.
Each should be labelled with a summary.


And a jump needs to verify B1_exit => B2_entry
And a block needs to verify forall n, B1_entry_n => wp(   )
where we put an assert on each exit.




In [ ]:
# or perhaps just have the last instruction be 
class Def:
    dest:
    comp:
    # jmp is Null.

# should check out angr and/or the fuzzing book.
    
class BasicBlock:
    defs: list[Def]
    jmp: Instr # no Null allowed though.

        
        

class BasicBlock():
    def __init__(self, code):
        self.body =
        self.jmp = 
def blocks(instrs):
    blocks = []
    defs = []
    for instr in instrs:
        if instr.jmp != Jump.NULL:
            blocks.append(Block(defs,instr))
            defs = []
        else:
            defs.append(instr)
    assert length(defs) == 0
    return blocks